In [1]:
import joblib
import pandas as pd
import colorama as col
from matplotlib import pyplot as plt
import ffn
import numpy as np
import datetime as dt
import fxcmpy
from tqdm import tqdm
import sys
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report, f1_score
sys.path.append('../') 
import pyttsx3
engine = pyttsx3.init()
from slack_sdk import WebClient
_slack_token = joblib.load('TOKENS/_slack_token.dag')
client = WebClient(token=_slack_token)
import time
import warnings
import talib
#from pycaret.regression import *
from sklearn.linear_model import LinearRegression

In [2]:
df = joblib.load('BASES/EURUSD_m5')
df_H1 = joblib.load('BASES/EURUSD_H1')
df_D1 = joblib.load('BASES/EURUSD_D1')
df_W1 = joblib.load('BASES/EURUSD_W1')
x = 'EUR/USD'
_period = 'm5'
_period2 = 'H1'
_ticker = x.replace('/','')
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2016-06-30' # stop the train and begin the test there '2016-08-31'
_stop = '2017-12-31' # stop the test there. After that, it is kept for oos '2017-12-31'
_last = '2021-04-09'

In [3]:
df_D1.dropna()

,Open,High,Low,Close,Symbol,OpenS,HighS,LowS,CloseS,Date,ADR
Lindex,,,,,,,,,,,
2010-04-20,1.348635,1.352270,1.341915,1.342235,EURUSD,1.348781,1.355012,1.343648,1.349788,2010-04-20,0.011005
2010-04-21,1.342235,1.344725,1.335845,1.338805,EURUSD,1.341893,1.347760,1.334891,1.342094,2010-04-21,0.010575
2010-04-22,1.338805,1.342150,1.320215,1.322305,EURUSD,1.338607,1.344625,1.332698,1.339250,2010-04-22,0.010273
2010-04-23,1.322305,1.339970,1.320585,1.337365,EURUSD,1.322204,1.327283,1.312599,1.321542,2010-04-23,0.011027
2010-04-26,1.336275,1.341610,1.329145,1.339475,EURUSD,1.340340,1.346025,1.331834,1.339499,2010-04-26,0.011858
...,...,...,...,...,...,...,...,...,...,...,...
2021-04-19,1.196895,1.204795,1.194250,1.204265,EURUSD,1.198084,1.201362,1.195048,1.198865,2021-04-19,0.005787
2021-04-20,1.204275,1.207980,1.202245,1.203250,EURUSD,1.204283,1.206602,1.199654,1.202243,2021-04-20,0.006092
2021-04-21,1.203245,1.204370,1.199835,1.203730,EURUSD,1.203223,1.205853,1.198488,1.202007,2021-04-21,0.006103


In [4]:
df[['Date'	,'ADR']][df.index=='2021-04-22']

,Date,ADR
2021-04-22,2021-04-22,0.005926


In [5]:
df[df.index=='2021-04-22 00:00:00']

,Close,CloseAsk,CloseBid,High,HighAsk,HighBid,Low,LowAsk,LowBid,Open,...,HighShift_weekly,LowShift_weekly,CloseShift_weekly,OpenShift_daily,HighShift_daily,LowShift_daily,CloseShift_daily,DateIndex,RSI_2_hourly,RSI_14_hourly
2021-04-22,1.20385,1.20391,1.20379,1.203995,1.20406,1.20393,1.20371,1.20376,1.20366,1.203735,...,1.204795,1.190925,1.196941,1.203479,1.207186,1.200867,1.204989,2021-04-22,85.117875,58.217236


In [10]:
%%time
# Debut de bougie daily
_condition_1 = df['Date'] != df['Date'].shift()

# Amorce
df['HighSlope'] = np.where(_condition_1,df['High'], 10000)

# Fin de bougie daily
_condition_2 = df['Date'] != df['Date'].shift(-1)
# Intérieur de bouige
_condition_3 = (df['Date'] == df['Date'].shift()) & (df['Date'] == df['Date'].shift(-1))
# Condition des highs et low par rapport à la bougie précédente
_condition_4 = df['High'] <= df['HighSlope'].shift(1)
_condition_5 = df['High'] > df['HighSlope'].shift(1)
#_condition_6 = df['Low'] < df['LowSlope'].shift()
#_condition_7 = df['Low'] >= df['LowSlope'].shift()

for i in range(1,len(df)-1):
    
    if (df.Date[i] == df.Date[i-1]) and (df.Date[i] == df.Date[i+1]) and df.High[i] <= df.HighSlope[i-1]:
        df.loc[df.index[i],'HighSlope'] =  df.HighSlope[i-1]
    
    elif (df.Date[i] == df.Date[i-1]) and (df.Date[i] == df.Date[i+1]) and df.High[i] > df.HighSlope[i-1]:
        df.loc[df.index[i],'HighSlope'] =  df.High[i]

CPU times: user 34min 6s, sys: 4min, total: 38min 6s
Wall time: 38min 8s


In [11]:
10000 in df['HighSlope'].tolist()

True

In [12]:
df[['Date','High','HighSlope']][df.HighSlope==10000].shape,df.shape

((2933, 3), (819803, 66))

In [20]:
df[['Date','High','HighSlope']][(df.index>='2021-04-15 23:45:00')&(df.index<='2021-04-16 00:35:00')]

,Date,High,HighSlope
2021-04-15 23:45:00,2021-04-15,1.197020,1.199325
2021-04-15 23:50:00,2021-04-15,1.197010,1.199325
2021-04-15 23:55:00,2021-04-15,1.196955,10000.000000
2021-04-16 00:00:00,2021-04-16,1.197275,1.197275
2021-04-16 00:05:00,2021-04-16,1.197475,1.197475
2021-04-16 00:10:00,2021-04-16,1.197415,1.197475
2021-04-16 00:15:00,2021-04-16,1.197205,1.197475
2021-04-16 00:20:00,2021-04-16,1.196740,1.197475
2021-04-16 00:25:00,2021-04-16,1.196540,1.197475
2021-04-16 00:30:00,2021-04-16,1.196330,1.197475


In [36]:
%%time
# Debut de bougie daily
_condition_1 = df['Date'] != df['Date'].shift()

# Amorce
df['HighSlope'] = np.where(_condition_1,df['High'], 10000)

# Fin de bougie daily
_condition_2 = df['Date'] != df['Date'].shift(-1)
# Intérieur de bouige
_condition_3 = (df['Date'] == df['Date'].shift()) & (df['Date'] == df['Date'].shift(-1))
# Condition des highs et low par rapport à la bougie précédente
_condition_4 = df['High'] <= df['HighSlope'].shift(1)
_condition_5 = df['High'] > df['HighSlope'].shift(1)
#_condition_6 = df['Low'] < df['LowSlope'].shift()
#_condition_7 = df['Low'] >= df['LowSlope'].shift()

df['HighSlope'] = np.where(_condition_3 & _condition_4,df.HighSlope.shift(1),df.HighSlope)
df['HighSlope'] = np.where(_condition_3 & _condition_5,df.HighSlope,df.HighSlope)
df[['Date','High','HighSlope']][df.HighSlope==10000].shape,df.shape

CPU times: user 67.8 ms, sys: 8.44 ms, total: 76.2 ms
Wall time: 62.1 ms


((815218, 3), (819803, 66))

In [37]:
df[['Date','High','HighSlope']][(df.index>='2021-04-15 23:45:00')&(df.index<='2021-04-16 00:35:00')]

,Date,High,HighSlope
2021-04-15 23:45:00,2021-04-15,1.197020,10000.000000
2021-04-15 23:50:00,2021-04-15,1.197010,10000.000000
2021-04-15 23:55:00,2021-04-15,1.196955,10000.000000
2021-04-16 00:00:00,2021-04-16,1.197275,1.197275
2021-04-16 00:05:00,2021-04-16,1.197475,10000.000000
2021-04-16 00:10:00,2021-04-16,1.197415,10000.000000
2021-04-16 00:15:00,2021-04-16,1.197205,10000.000000
2021-04-16 00:20:00,2021-04-16,1.196740,10000.000000
2021-04-16 00:25:00,2021-04-16,1.196540,10000.000000
2021-04-16 00:30:00,2021-04-16,1.196330,10000.000000


In [30]:
2933/819803

0.003577688786208394

In [34]:
df[['Date','High','HighSlope']][df.HighSlope==10000].shape,df.shape

((359815, 3), (819803, 66))

In [5]:
%%time
_flagh = 0
_flagl = 0
val = 0

def fh(row):
    global _flagh, val
   
    if row['High'] < row['DayHigh'] and row['High'] <= row['HighSlope'].shift(1) and _flagh == 0 and row['Date'] == row['DateShiftMinus'] and row['Date'] == row['DateShiftPlus']:
        val = row['HighSlope'].shift(1) # np.nan
        _flagh = 0
    
    elif row['High'] < row['DayHigh'] and row['High'] >= row['HighSlope'].shift(1) and _flagh == 0 and row['Date'] == row['DateShiftMinus'] and row['Date'] == row['DateShiftPlus']:
        val = row['High'] # 
        _flagh = 0
    
    elif row['High'] >= row['DayHigh'] and row['High'] >= row['HighSlope'].shift(1) and _flagh == 0 and row['Date'] == row['DateShiftMinus'] and row['Date'] == row['DateShiftPlus']:
        val = row['DayHigh']
        _flagh = 1
        
    
    #elif _flagh == 1:
     #   val = row['DayHigh']

            
    return(val)

def fl(row):
    global _flagl, val
    
    if row['Low'] > row['DayLow'] and row['Low'] >= row['LowSlope'].shift(1) and _flagl == 0 and row['Date'] == row['DateShiftMinus'] and row['Date'] == row['DateShiftPlus']:
        _flagl = 0
        val =  row['LowSlope'].shift(1) # np.nan
    
    elif row['Low'] > row['DayLow'] and row['Low'] <= row['LowSlope'].shift(1) and _flagl == 0 and row['Date'] == row['DateShiftMinus'] and row['Date'] == row['DateShiftPlus']:
        val = row['Low']
        _flagl = 0
    
    elif row['Low'] <= row['DayLow'] and row['Low'] <= row['LowSlope'].shift(1) and _flagl == 0 and row['Date'] == row['DateShiftMinus'] and row['Date'] == row['DateShiftPlus']:
        val = row['DayLow']
        _flagl = 1

    #elif _flagh == 1:
     #   val = row['DayLow']

    return(val)



CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 10 µs


In [ ]:
%%time
df['DateShiftMinus'] = df.Date.shift(1)
df['DateShiftPlus'] = df.Date.shift(-1)

df['HighShift'] = df.High.shift(1)
df['LowShift'] = df.Low.shift(1)

df['HighSlope'] = np.where(df['Date'] != df['Date'].shift(),df['High'], np.nan)
df['LowSlope'] = np.where(df['Date'] != df['Date'].shift(),df['Low'], np.nan)

df['DayHigh'] = df['High'].groupby(df['Date']).max()
df['DayHigh'].fillna(method='ffill', inplace=True)
df['DayLow'] = df['Low'].groupby(df['Date']).min()
df['DayLow'].fillna(method='ffill', inplace=True)


df['HighSlope'] = df.apply(fh,axis=1)
df['LowSlope'] = df.apply(fl,axis=1)

#df['ADR_High'] = df.LowSlope + df.ADR
#df['ADR_Low'] = df.HighSlope - df.ADR

df[['Date','DateShiftMinus','DateShiftPlus','High','Low','HighShift','LowShift','DayLow','DayHigh','HighSlope','LowSlope','ADR_High','ADR_Low']][(df.index>='2021-04-19 23:50:00')&(df.index<='2021-04-20 02:10:00')]

In [13]:
%%time

df['ADR_High'] = df.Low + df.ADR
df['ADR_Low'] = df.High - df.ADR

# Intérieur de bouige
_condition_1 = (df['Date'] == df['Date'].shift(1)) & (df['Date'] == df['Date'].shift(-1))
_condition_2 = df.ADR_High >= df.ADR_High.shift(1)
_condition_3 = df.ADR_Low <= df.ADR_Low.shift(1)

df['ADR_High'] = np.where(_condition_1 & _condition_2,df.ADR_High.shift(1),df.ADR_High.shift(1))
#df['ADR_Low'] = np.where(_condition_1 & _condition_3,df.ADR_Low.shift(1),df.ADR_Low.shift(1))

df[['Date','High','Low','ADR','ADR_High','ADR_Low']][(df.index>='2021-04-19 23:50:00')&(df.index<='2021-04-20 02:10:00')]

CPU times: user 63.9 ms, sys: 15.6 ms, total: 79.5 ms
Wall time: 58.3 ms


,Date,High,Low,ADR,ADR_High,ADR_Low
2021-04-19 23:50:00,2021-04-19,1.204320,1.204105,0.005787,1.210007,1.198533
2021-04-19 23:55:00,2021-04-19,1.204370,1.204165,0.005787,1.209892,1.198583
2021-04-20 00:00:00,2021-04-20,1.204420,1.204165,0.006092,1.209952,1.198327
2021-04-20 00:05:00,2021-04-20,1.204525,1.204295,0.006092,1.210257,1.198433
2021-04-20 00:10:00,2021-04-20,1.204465,1.204280,0.006092,1.210387,1.198373
2021-04-20 00:15:00,2021-04-20,1.204555,1.204380,0.006092,1.210373,1.198463
2021-04-20 00:20:00,2021-04-20,1.204515,1.204290,0.006092,1.210472,1.198422
2021-04-20 00:25:00,2021-04-20,1.204410,1.204280,0.006092,1.210382,1.198317
2021-04-20 00:30:00,2021-04-20,1.204360,1.204250,0.006092,1.210373,1.198268
2021-04-20 00:35:00,2021-04-20,1.204370,1.204250,0.006092,1.210342,1.198278


In [26]:
1.204165+	0.006092

1.210257

In [4]:
df[['HighSlope'	,'LowSlope']].shape,df[['HighSlope'	,'LowSlope']].dropna().shape

((819949, 2), (2933, 2))

In [5]:
df[['Date','DateShiftMinus','DateShiftPlus','High','Low','HighShift','LowShift','DayLow','DayHigh','HighSlope','LowSlope','ADR_High','ADR_Low']].head(60)

,Date,DateShiftMinus,DateShiftPlus,High,Low,HighShift,LowShift,DayLow,DayHigh,HighSlope,LowSlope,ADR_High,ADR_Low
2010-01-04 00:00:00,2010-01-04,NaT,2010-01-04,1.431640,1.430370,NaN,NaN,1.42577,1.44556,1.43164,1.43037,NaN,NaN
2010-01-04 00:05:00,2010-01-04,2010-01-04,2010-01-04,1.431130,1.429600,1.431640,1.430370,1.42577,1.44556,NaN,NaN,NaN,NaN
2010-01-04 00:10:00,2010-01-04,2010-01-04,2010-01-04,1.430840,1.429050,1.431130,1.429600,1.42577,1.44556,NaN,NaN,NaN,NaN
2010-01-04 00:15:00,2010-01-04,2010-01-04,2010-01-04,1.430270,1.428970,1.430840,1.429050,1.42577,1.44556,NaN,NaN,NaN,NaN
2010-01-04 00:20:00,2010-01-04,2010-01-04,2010-01-04,1.431210,1.429920,1.430270,1.428970,1.42577,1.44556,NaN,NaN,NaN,NaN
2010-01-04 00:25:00,2010-01-04,2010-01-04,2010-01-04,1.432020,1.431015,1.431210,1.429920,1.42577,1.44556,NaN,NaN,NaN,NaN
2010-01-04 00:30:00,2010-01-04,2010-01-04,2010-01-04,1.431590,1.431030,1.432020,1.431015,1.42577,1.44556,NaN,NaN,NaN,NaN
2010-01-04 00:35:00,2010-01-04,2010-01-04,2010-01-04,1.431160,1.430570,1.431590,1.431030,1.42577,1.44556,NaN,NaN,NaN,NaN
2010-01-04 00:40:00,2010-01-04,2010-01-04,2010-01-04,1.430760,1.429605,1.431160,1.430570,1.42577,1.44556,NaN,NaN,NaN,NaN
2010-01-04 00:45:00,2010-01-04,2010-01-04,2010-01-04,1.430725,1.429960,1.430760,1.429605,1.42577,1.44556,NaN,NaN,NaN,NaN


In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 819803 entries, 2010-01-04 00:00:00 to 2021-04-23 08:40:00
Data columns (total 71 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Close              819803 non-null  float64       
 1   CloseAsk           819803 non-null  float64       
 2   CloseBid           819803 non-null  float64       
 3   High               819803 non-null  float64       
 4   HighAsk            819803 non-null  float64       
 5   HighBid            819803 non-null  float64       
 6   Low                819803 non-null  float64       
 7   LowAsk             819803 non-null  float64       
 8   LowBid             819803 non-null  float64       
 9   Open               819803 non-null  float64       
 10  OpenAsk            819803 non-null  float64       
 11  OpenBid            819803 non-null  float64       
 12  Symbol             819803 non-null  object        
 13  Date      